In [ ]:
pip install "python_louvain==0.13"


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install progressbar


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import time
import progressbar

In [ ]:
import community
import networkx as nx
from tqdm import tqdm


In [ ]:

class EdMot(object):
    def __init__(self, graph, component_count, cutoff):
        self.graph = graph
        self.component_count = component_count
        self.cutoff = cutoff

    def _overlap(self, node_1, node_2):
        nodes_1 = self.graph.neighbors(node_1)
        nodes_2 = self.graph.neighbors(node_2)
        return len(set(nodes_1).intersection(set(nodes_2)))

    def _calculate_motifs(self):
        print("\nCalculating overlaps.\n")
        edges = [e for e in tqdm(self.graph.edges()) if self._overlap(e[0], e[1]) >= self.cutoff]
        self.motif_graph = nx.from_edgelist(edges)

    def _extract_components(self):
        print("\nExtracting components.\n")
        components = [c for c in nx.connected_components(self.motif_graph)]
        components = [[len(c), c] for c in components]
        components.sort(key=lambda x: x[0], reverse=True)
        important_components = [components[comp][1] for comp
                                in range(self.component_count if len(components)>=self.component_count else len(components))]
        self.blocks = [list(graph) for graph in important_components]

    def _fill_blocks(self):
        print("Adding edge blocks.\n")
        bar = progressbar.ProgressBar().
        new_edges = [(n_1, n_2) for nodes in self.blocks for n_1 in nodes for n_2 in nodes if n_1!= n_2]
        self.graph.add_edges_from(new_edges)
        print("Blocks completed")

    def fit(self):
        self._calculate_motifs()
        self._extract_components()
        self._fill_blocks()
        bar = progressbar.ProgressBar()
        partition = community.best_partition(self.graph)
        return partition


In [ ]:
import json
import pandas as pd
import networkx as nx

def graph_reader(path):
    graph = nx.from_edgelist(pd.read_csv(path).values.tolist())
    graph.remove_edges_from(nx.selfloop_edges(graph))
    return graph

def membership_saver(membership_path, membership):
    with open(membership_path, "w") as f:
        json.dump(membership, f)


In [ ]:

def main():

    graph = graph_reader("./input/edges_1.csv")
    model = EdMot(graph, 2, 50)
    memberships = model.fit()
    membership_saver("./output/facebook_membership.json", memberships)
    
if __name__ == "__main__":
    main()
